In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

'02.17.2022'

In [2]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('FanDuel-NBA*.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [40]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [41]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [42]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Dennis Schroder,HOU,LAC,35.0,PG/SG,4300.0,31.2,9.0,40.6,66.6,MAIN,Dennis Schroder,PG/SG,71761-40201,G,NaN,PG/SG/G
1,Seth Curry,BKN,WAS,34.5,PG/SG,6000.0,35.3,6.3,43.4,36.6,MAIN,Seth Curry,PG/SG,71761-15583,G,NaN,PG/SG/G
2,Kyle Lowry,MIA,CHA,36.0,PG,5600.0,33.2,5.8,41.8,32.2,MAIN,Kyle Lowry,PG,71761-9535,G,NaN,PG/G
3,Luka Doncic,DAL,NOP,37.0,PG/SG,11300.0,55.7,5.5,66.1,36.8,MAIN,Luka Doncic,PG/SG,71761-84669,G,NaN,PG/SG/G
4,Kyle Kuzma,WAS,BKN,36.0,SF/PF,7700.0,40.8,5.0,49.2,52.0,MAIN,Kyle Kuzma,SF/PF,71761-58377,NaN,F,SF/PF/F


In [43]:
df.to_excel("FD_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)

df = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ str(today) + ".xlsx")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Dennis Schroder,HOU,LAC,35.0,PG/SG,4300,31.2,9.0,40.6,66.6,MAIN,Dennis Schroder,PG/SG,71761-40201,G,NaN,PG/SG/G
1,Seth Curry,BKN,WAS,34.5,PG/SG,6000,35.3,6.3,43.4,36.6,MAIN,Seth Curry,PG/SG,71761-15583,G,NaN,PG/SG/G
2,Kyle Lowry,MIA,CHA,36.0,PG,5600,33.2,5.8,41.8,32.2,MAIN,Kyle Lowry,PG,71761-9535,G,NaN,PG/G
3,Luka Doncic,DAL,NOP,37.0,PG/SG,11300,55.7,5.5,66.1,36.8,MAIN,Luka Doncic,PG/SG,71761-84669,G,NaN,PG/SG/G
4,Kyle Kuzma,WAS,BKN,36.0,SF/PF,7700,40.8,5.0,49.2,52.0,MAIN,Kyle Kuzma,SF/PF,71761-58377,NaN,F,SF/PF/F


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [44]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [45]:
# number of players constraint
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['list'].iloc[i]) for i in range(len(df))])
    
# prob += get_position_sum(player_vars, df, 'PG') <= 1
prob += get_position_sum(player_vars, df, 'PG') >= 2
# prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 2
prob += get_position_sum(player_vars, df, 'G') >= 4
# prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
# prob += get_position_sum(player_vars, df, 'PF') <= 2
prob += get_position_sum(player_vars, df, 'PF') >= 2
prob += get_position_sum(player_vars, df, 'F') >= 4
prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1


# total salary constraint
prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [46]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [47]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['list'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

PG/SG/G Dennis Schroder HOU 4300 31.2
PG/SG/G Seth Curry BKN 6000 35.3
PG/G Kyle Lowry MIA 5600 33.2
PG/SG/G Luka Doncic DAL 11300 55.7
SF/PF/F Kyle Kuzma WAS 7700 40.8
C Jonas Valanciunas NOP 6400 35.5
SF/SG/G/F Jimmy Butler MIA 9100 46.1
PF/F Jaxson Hayes NOP 4400 25.6
SF/PF/F Jae'Sean Tate HOU 5200 27.0


(60000, 330.40000000000003)